In [1]:
import PyPDF2
import os
import re
import requests
import pandas as pd
from itertools import combinations 
from itertools import permutations
from itertools import chain
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
import numpy as np
ps = PorterStemmer()

In [2]:
courseletlist = (
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F175%2F163757167020201210_Liu_crypto_p2p_lending.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F272%2F1654160257Lesson1-1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F273%2F1654160288Lesson1-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F274%2F1654160327Lesson1-3.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F275%2F1654160374Lesson1-4.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F276%2F1654160475Lesson1-5.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F277%2F1654160518Lesson1-6.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F279%2F1654251498Lesson2-1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F278%2F1654160549Lesson1-7.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F280%2F1654251511Lesson2-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F281%2F1654251525Lesson2-3.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F103%2F20210303+IA+METIS+Reinforcement+Learning.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F107%2F1636712642CATE_meets_ML_Presentation.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F120%2F163458263420190429+Hae+Ni+LDA+DTM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F120%2F163646337920210921+Hae+Ni+LDA.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F121%2F163646358220210708+Hae+Ni+LDA+extensions.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F108%2F163595835420210530+METIS+WANG+Kalman+Filter.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F110%2F1632126441nodalida2021_summaryQuality_slides.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F112%2F163664661320211013+Ren+LI+Hae+Expectile+FRM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F114%2F1635233254Shapley.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F118%2F1636625638FRM%40EM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F119%2F163231165920210324+Wan+Hae+Li+k-expectile+clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F130%2F1633104997PAC.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F134%2F163368764620210923+Mer+Hae+GAN+Generative+Adversarial+Networks.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F139%2F163402565120211012+Kho+Hae+Trespassing+random+forests+with+a+pointed+stick+for+self+defence.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F217%2F1644582711Berlin_short_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F235%2F1649426301Variable+importance+measures+for+RF+.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F179%2F16376558632021122+SBA+JW+Hae+EPF++Quantinar.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F186%2F1645194357Presentation_Quantinar_with_videos.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F306%2FBarHan2021_talk.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F181%2F163826827620211130+LI+Hae+Case+based+Bancruptcy+prediction.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F224%2F164728525020220305+LI+Electricity+Market+Coupling.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F196%2F1642599075163458186420200403+METIS+Kho+Hae+Spectral+Clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F197%2F1642599236164171878820211207+Hae+Zin+Hierarchical+Clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F198%2F1642599289163231165920210324+Wan+Hae+Li+k-expectile+clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F200%2F1642686124163527012720210526+SAE+NAG+HAE+SIZ+Understanding+jumps+in+high+frequency+digital+asset+markets_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F201%2F16426862241636625638FRM%40EM_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F202%2F1642686343163774999520210912+Hae+Li+Tao+Dynamic+Crypto+Networks_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F189%2F163958119120211130+Hae+Wan+Kot+ComputerMuseum.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F183%2F1643806658KDE+ill-posed+problems.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F167%2F163699399420211115+Liu+Word+Embeddings+2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F157%2F1642778303introduction_data_science.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F180%2F16377653596.+model+assessment+-+part+4+-+appendix.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F128%2F163707553520210331_METIS_Hel_GANs_for_Time_Series.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F127%2F163458156320190528+Cea+Hae+Scagnostics.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F117%2F163458186420200403+METIS+Kho+Hae+Spectral+Clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F106%2F163458175020200914+Hae+DS2+Data+Science+%26+Digital+Society.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F190%2F1640038524Instruction+for+Creating+Quantlets.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F144%2F1636624210NNCSR_Slides.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F184%2F163888970320211207+Zin+Reu+Hae+USC+Quantinar+40+min+PDF.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F133%2F163707963820210525_Hae_Xia_Crypto_Indices-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F129%2F163458163120200915+Kim+Hae+Tri+VCRIX.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F123%2F163664570620210922+Mat+Pac+Hae+guide+hedging+CC.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F122%2F163283000220210923+Cul+Hae+Pet+Xia+Cryptocurrency+as+an+asset+class.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F116%2F163774999520210912+Hae+Li+Tao+Dynamic+Crypto+Networks.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F113%2F1632580703FRM+for+Cryptos.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F101%2F163170747120210914+Reu+DSF+Digital+Surrogate+Finance+Doc.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F97%2F163458105020210808+METIS+Win+Pricing+Kernel+Risk+Premium.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F96%2F163299370720210908_CRC21_Hae_Rodeo_or_Ascot.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F96%2F1635155323202109_RoA.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F79%2F164604019720210502+Hae+Har+Reu+Understanding+CCs.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F241%2F1650632942Biographical+Background+Information.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F210%2F164390414020220130+METIS+Gua+Hae+Model+Selection+Criteria.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F194%2F164171878820211207+Hae+Zin+Hierarchical+Clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F170%2F163709451820211117+Hae+Qia+Network+Centrality.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F142%2F163627928020211107+Hae+Iva+Mat+Delaunay+Triangulation_A_Shape.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F135%2F1649084960Chapter+1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F136%2F1649094328Chapter+2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F137%2F1633950248Chapter+3.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F138%2F1649094430Chapter+4.pdf'
)

ids = (["id-{}".format(i+1) for i in range(len(courseletlist))])


stopwords_list = {"i","me","my","myself","we","our","ours","ourselves","you","you're","you've","you'll","you'd","your","yours","yourself","yourselves","he","him","his","himself","she","she's","her","hers","herself","it","it's","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this",'that',"that'll","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","don't","should","should've","now","d","ll","m","o","re","ve","y","ain","aren","aren't","couldn","couldn't","didn","didn't","doesn","doesn't","hadn","hadn't","hasn","hasn't","haven","haven't","isn","isn't","ma","mightn","mightn't","mustn","mustn't","needn","needn't","shan","shan't","shouldn","shouldn't","wasn","wasn't","weren","weren't","won","won't","wouldn","wouldn't"}

In [3]:
def download_pdf(file_name, url):
    '''Download a PDF file with an URL (Step 1)'''
    # Define HTTP Headers
    headers = {"User-Agent": "Chrome/51.0.2704.103"}
    # Download image
    response = requests.get(url, headers=headers)
    # if response is OK download the PDF and store it, else write the status
    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
    else:
        print(response.status_code)
    return

In [4]:
def create_string(file_name):
    '''Transform a PDF file to a list of string pages (Step 2)'''
    # opening the file
    imported_pdf = open(file_name, 'rb')
    # convert PDF to readable file
    transformed_pdf = PyPDF2.PdfFileReader(imported_pdf)
    # get number of pages
    totalpages = transformed_pdf.numPages
    # read the data and store in a list
    pdf_output = [transformed_pdf.getPage(i) for i in range(totalpages)]
    # extract result
    pdf_output = [pdf_output[i].extractText() for i in range(totalpages)]
    return pdf_output, totalpages 

In [5]:
def cleaning(file_name):
    '''Initial PDF cleaning procedure (Step 3)'''
    pdf_output, totalpages = create_string(file_name)
    # # cleaning URLs
    pdf_output = [re.sub(pattern = "http[^ ]*", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    # # cleaning symbols
    pdf_output = [re.sub(pattern = "\\n", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = "\W|\d", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = "[^a-zA-Z]", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    # # cleaning multispaces
    pdf_output = [re.sub(pattern = "\s{2,}", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    # # cleaning out 1-2-worders
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    # # lower-casing
    pdf_output = [pdf_output[i].lower() for i in range(totalpages)]
    return pdf_output, totalpages

In [6]:
def word_lists(file_name):
    '''Creating the base one-word, two-word and three-word lists, the permutation lists for two- and three-word lists (Step 4)'''
    pdf_output, totalpages = cleaning(file_name)
    # split to a list
    word_list = [pdf_output[i].split(" ") for i in range(totalpages)]
    # stemming
    word_list_stemmed = [[ps.stem(word_list[i][j]) for j in range(len(word_list[i]))] for i in range(totalpages)]    
    word_list_stemmed = pd.DataFrame(word_list_stemmed)
    # one-word section
    one_word_list = [word_list_stemmed.iloc[j, i] for j in range(totalpages) for i in range(len(word_list_stemmed))]
    # one_word_list = [x for x in one_word_list if len(x) > 2] # problem with nulls
    one_word_list = [x for x in one_word_list if x not in stopwords_list]

    # two-word section
    two_word_list = [[word_list_stemmed.iloc[j, i], word_list_stemmed.iloc[j, i+1]] for j in range(totalpages)  for i in range(len(word_list_stemmed) - 1)]
    two_word_permutation_list = [[p for p in permutations(two_word_list[k])][1:] for k in range(len(two_word_list))]
    two_word_permutation_set = set(list(chain(*two_word_permutation_list)))
    two_word_permutation_set = pd.DataFrame(two_word_permutation_set)

    # three-word section
    three_word_list = [[word_list_stemmed.iloc[j, i], word_list_stemmed.iloc[j, i+1], word_list_stemmed.iloc[j, i+2]] for j in range(totalpages) for i in range(len(word_list_stemmed) - 2)]
    three_word_permutation_list = [[p for p in permutations(three_word_list[k])][1:] for k in range(len(three_word_list))]
    three_word_permutation_set = set(list(chain(*three_word_permutation_list)))
    three_word_permutation_set = pd.DataFrame(three_word_permutation_set)

    return word_list_stemmed, one_word_list, two_word_list, two_word_permutation_list, two_word_permutation_set, three_word_list, three_word_permutation_list, three_word_permutation_set

In [10]:
def occurance_one_matrix_creator(file_name):
    '''Creating the occurrance matrix for one-word combinations (Step 7)'''
    word_list_stemmed, one_word_list, two_word_list, two_word_permutation_list, two_word_permutation_set, three_word_list, three_word_permutation_list, three_word_permutation_set = word_lists(file_name)
    # copying the data
    words = one_word_list.copy()
    # creating the three-word combinations as one string
    words = [x for x in words if x != ""]
    words = [x for x in words if x != " "]    
    # crating the dictionary
    dictionary_one_word = {}
    
    # counting word occurances
    for word in words:
        if word in dictionary_one_word:
            dictionary_one_word[word] = dictionary_one_word[word] + 1
        else:
            dictionary_one_word[word] = 1
    dictionary_one_word = list(dictionary_one_word)
    dictionary_one_word = list(filter(None, dictionary_one_word))
    return dictionary_one_word

In [11]:
download_pdf(ids[0], courseletlist[0])

pdf_output, totalpages = cleaning(ids[0])

word_list_stemmed, one_word_list, two_word_list, two_word_permutation_list, two_word_permutation_set, three_word_list, three_word_permutation_list, three_word_permutation_set = word_lists(ids[0])

dictionary_one_word = occurance_one_matrix_creator(ids[0])

combined_pdf = [' '.join(pdf_output)]

In [12]:
for i in range(1,20):
    download_pdf(ids[i], courseletlist[i])
    pdf_output, totalpages = cleaning(ids[i])
    word_list_stemmed, one_word_list, two_word_list, two_word_permutation_list, two_word_permutation_set, three_word_list, three_word_permutation_list, three_word_permutation_set = word_lists(ids[i])
    dictionary_one_word_0 = occurance_one_matrix_creator(ids[i])
    [dictionary_one_word.append(j) for j in dictionary_one_word_0]
    combined_pdf_0 = [' '.join(pdf_output)]
    combined_pdf.extend(combined_pdf_0)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_one = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase = False, stop_words = stopwords_list, ngram_range=(1,1))
X_one = vectorizer_one.fit_transform(combined_pdf)

vectorizer_two = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase = False, stop_words = stopwords_list, ngram_range=(2,2))
X_two = vectorizer_two.fit_transform(combined_pdf)

vectorizer_three = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase = False, stop_words = stopwords_list, ngram_range=(3,3))
X_three = vectorizer_three.fit_transform(combined_pdf)

In [15]:
# The tuple represents: (document_id, token_id)
# The value following the tuple represents the tf-idf score of a given token in a given document
# The tuples that are not there have a tf-idf score of 0
# vectorizer_one.get_feature_names_out()[5278]

print(X_one)

  (0, 5278)	0.03228357851378033
  (0, 370)	0.025606537088060658
  (0, 1742)	0.03228357851378033
  (0, 3903)	0.03228357851378033
  (0, 466)	0.03228357851378033
  (0, 1659)	0.025606537088060658
  (0, 4890)	0.04804772376566294
  (0, 947)	0.06775071459120116
  (0, 3028)	0.03228357851378033
  (0, 1394)	0.02685644950533422
  (0, 35)	0.023457009862394726
  (0, 3601)	0.03228357851378033
  (0, 193)	0.03228357851378033
  (0, 655)	0.020215794828223357
  (0, 2951)	0.03228357851378033
  (0, 3842)	0.03228357851378033
  (0, 2497)	0.028377759663972598
  (0, 2287)	0.03228357851378033
  (0, 3494)	0.028377759663972598
  (0, 2566)	0.03228357851378033
  (0, 4031)	0.028377759663972598
  (0, 3832)	0.04804772376566294
  (0, 1589)	0.028377759663972598
  (0, 691)	0.04554139396772176
  (0, 2971)	0.0546608499389928
  :	:
  (19, 1748)	0.05107421570459286
  (19, 420)	0.029249595365229173
  (19, 527)	0.031805821283743395
  (19, 361)	0.03909800025140455
  (19, 4735)	0.01916726519193831
  (19, 3162)	0.0154649204385193

In [17]:
from sklearn.decomposition import TruncatedSVD

lsa_obj = TruncatedSVD(n_components=30)
tfidf_lsa_data_one = lsa_obj.fit_transform(X_one)
Sigma_one = lsa_obj.singular_values_
V_T_one = lsa_obj.components_.T

In [18]:
lsa_obj = TruncatedSVD(n_components=30)
tfidf_lsa_data_two = lsa_obj.fit_transform(X_two)
Sigma_two = lsa_obj.singular_values_
V_T_two = lsa_obj.components_.T

In [20]:
lsa_obj = TruncatedSVD(n_components=30)
tfidf_lsa_data_three = lsa_obj.fit_transform(X_three)
Sigma_three = lsa_obj.singular_values_
V_T_three = lsa_obj.components_.T

In [19]:
# generates U
tfidf_lsa_data_one

array([[ 3.53975607e-01, -6.40727289e-02,  1.77794030e-01,
         9.70925985e-02, -3.14102344e-01, -9.00257300e-02,
         8.52176421e-02, -9.17717972e-02,  4.03951053e-01,
         4.75012561e-01, -3.35510344e-01, -3.75676544e-01,
         6.19542474e-02,  1.66320861e-01, -1.28866834e-01,
        -1.13298204e-01, -5.30553369e-02,  1.55237445e-02,
        -2.18880472e-02, -1.08053688e-02],
       [ 3.72115694e-01, -2.31975195e-01, -2.03483552e-01,
        -1.17495863e-01, -1.97011073e-01, -2.70183208e-02,
         1.87823023e-01, -1.69592457e-01, -1.18494075e-01,
        -3.33540896e-01, -2.61950922e-01, -6.02601730e-02,
         5.68902822e-01, -2.77972396e-01,  1.43893416e-01,
        -2.46776333e-02, -1.65066138e-01,  4.29798843e-02,
        -7.48845184e-03, -8.01131351e-04],
       [ 3.96695536e-01, -2.24611864e-01, -1.85278997e-01,
        -1.16375128e-01, -1.94067563e-01, -4.23139722e-02,
         1.77262491e-01, -4.16596023e-02,  2.67915124e-01,
        -2.69510223e-01,  2.3

In [155]:
# generate Sigma.
# The singular values corresponding to each of the selected components. The singular values are equal to the 2-norms of the n_components variables in the lower-dimensional space.
Sigma

array([1.57868281, 1.27560806, 1.18931387, 1.06878374, 1.03176776,
       0.99565543, 0.97771788, 0.96309304, 0.9568175 , 0.95155689,
       0.93802241, 0.93250326, 0.91773359, 0.91421795, 0.89500509,
       0.84990666, 0.84338645, 0.82247837, 0.81541734, 0.74831367])

In [161]:
# The right singular vectors of the input data.
lsa_obj.components_

array([[ 0.0296059 ,  0.00408016,  0.00343791, ...,  0.01622122,
         0.00343791,  0.00343791],
       [ 0.07966318,  0.00927363,  0.00895464, ...,  0.04928592,
         0.00895464,  0.00895464],
       [-0.02484424, -0.00181574, -0.00213887, ..., -0.01457114,
        -0.00213887, -0.00213887],
       ...,
       [-0.01453849,  0.01854994,  0.02279379, ...,  0.01551424,
         0.02279379,  0.02279379],
       [ 0.00045156, -0.00573205, -0.00640671, ..., -0.00908877,
        -0.00640671, -0.00640671],
       [-0.02361227, -0.00044952, -0.00017779, ..., -0.06565155,
        -0.00017779, -0.00017779]])

In [156]:
# generates Sgenerates V_T
V_T

array([[ 0.0296059 ,  0.07966318, -0.02484424, ..., -0.01453849,
         0.00045156, -0.02361227],
       [ 0.00408016,  0.00927363, -0.00181574, ...,  0.01854994,
        -0.00573205, -0.00044952],
       [ 0.00343791,  0.00895464, -0.00213887, ...,  0.02279379,
        -0.00640671, -0.00017779],
       ...,
       [ 0.01622122,  0.04928592, -0.01457114, ...,  0.01551424,
        -0.00908877, -0.06565155],
       [ 0.00343791,  0.00895464, -0.00213887, ...,  0.02279379,
        -0.00640671, -0.00017779],
       [ 0.00343791,  0.00895464, -0.00213887, ...,  0.02279379,
        -0.00640671, -0.00017779]])

In [ ]:
print(lsa_obj.explained_variance_ratio_)

print(lsa_obj.explained_variance_ratio_.sum())

print(lsa_obj.singular_values_)

print(lsa_obj.feature_names_in_)

In [ ]:
# import seaborn as sns
# ax = sns.barplot(x=lsa_obj.feature_names_in_[0:20], y = Sigma[0:20])
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

In [ ]:
# term_topic_matrix = pd.DataFrame(data=lsa_term_topic, 
#                                  index = eda_train.columns, 
#                                  columns = [f'Latent_concept_{r}' for r in range(0,V_T.shape[1])])

